In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
from tensorflow.keras.applications.vgg19 import VGG19
import numpy as np
import pandas as pd
import PIL.Image as Image
import os
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.applications.vgg16 import VGG16

In [3]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Colab Notebooks/Breast Cancer/BUID Dataset.zip', 'r') as zipObj:

     zipObj.extractall('BreastCancer')

In [4]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import splitfolders
input_folder = '/content/BreastCancer/BUID/'

In [6]:
splitfolders.ratio(input_folder, output="dataset3",
                   seed=123, ratio=(.8, .1, .1), 
                   group_prefix=None)

Copying files: 780 files [00:00, 1588.76 files/s]


In [7]:
BATCH_SIZE = 64
IMAGE_SIZE = 224
CHANNELS=3
EPOCHS = 50

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/train',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/test',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/val',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

Found 623 files belonging to 3 classes.
Found 80 files belonging to 3 classes.
Found 77 files belonging to 3 classes.


In [9]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/dataset3/train')

class_names= train_dataset.class_names
class_names

Found 623 files belonging to 3 classes.


['benign', 'malignant', 'normal']

In [10]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [11]:
data_augmentation = tf.keras.Sequential([
layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
layers.experimental.preprocessing.RandomRotation(0.2),
])

In [12]:
training_ds = train_ds.map(
lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
VGG = keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')
VGG.output

58900480/58889256 [==============================] - 0s 0us/step


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [14]:
for layer in VGG.layers:
  VGG.trainable = False

In [15]:
x= Flatten()(VGG.output)
prediction=Dense(3,activation='softmax')(x)
model=Model(inputs=VGG.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
import time
start = time.time()
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)
print("Total time: ", time.time() - start, "seconds")

Epoch 1/50
10/10 [==============================] - 30s 1s/step - loss: 7.4298 - accuracy: 0.5650 - val_loss: 2.8598 - val_accuracy: 0.6623
Epoch 2/50
10/10 [==============================] - 7s 402ms/step - loss: 2.1450 - accuracy: 0.7657 - val_loss: 5.5818 - val_accuracy: 0.6104
Epoch 3/50
10/10 [==============================] - 7s 408ms/step - loss: 0.6160 - accuracy: 0.9133 - val_loss: 2.1354 - val_accuracy: 0.7532
Epoch 4/50
10/10 [==============================] - 7s 412ms/step - loss: 0.2173 - accuracy: 0.9535 - val_loss: 2.9542 - val_accuracy: 0.7922
Epoch 5/50
10/10 [==============================] - 7s 419ms/step - loss: 0.1509 - accuracy: 0.9647 - val_loss: 2.2754 - val_accuracy: 0.7922
Epoch 6/50
10/10 [==============================] - 7s 418ms/step - loss: 0.0759 - accuracy: 0.9872 - val_loss: 1.9500 - val_accuracy: 0.7403
Epoch 7/50
10/10 [==============================] - 7s 422ms/step - loss: 0.0237 - accuracy: 0.9904 - val_loss: 2.0514 - val_accuracy: 0.7662
Epoch 8/

In [19]:
scores = model.evaluate(test_ds)

2/2 [==============================] - 3s 2s/step - loss: 2.9580 - accuracy: 0.8125
